# 🔍 GradFaith-CAM Demo Notebook

**Seeing Isn’t Always Believing**  
Grad-CAM Faithfulness & Localization Analysis  
**KST 2026 | MARSAIL | Chulalongkorn University**

---
This notebook demonstrates:
1. Model loading
2. Grad-CAM generation
3. Faithfulness evaluation
4. Publication-quality visualization


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models, transforms
from PIL import Image

from gradcam.gradcam import GradCAM
from gradcam.faithfulness import perturbation_faithfulness

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

## 1️⃣ Load Pretrained Model
We use **ResNet-50** as in the paper experiments.

In [ ]:
model = models.resnet50(weights='IMAGENET1K_V2')
model.eval().to(device)

target_layer = model.layer4[-1]
gradcam = GradCAM(model, target_layer)

## 2️⃣ Load and Preprocess Image

In [ ]:
img_path = 'sample.jpg'  # replace with CT image
img = Image.open(img_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

x = transform(img).unsqueeze(0).to(device)

## 3️⃣ Prediction & Grad-CAM Generation

In [ ]:
with torch.no_grad():
    logits = model(x)
    pred_class = logits.argmax(1).item()

cam = gradcam(x, pred_class).squeeze()

print('Predicted class:', pred_class)

## 4️⃣ Faithfulness Evaluation (Perturbation Test)

In [ ]:
faith = perturbation_faithfulness(model, x, cam, pred_class)
print(f'Faithfulness score: {faith:.4f}')

## 5️⃣ Visualization (Publication Quality)

In [ ]:
import cv2

img_np = x.squeeze().permute(1,2,0).cpu().numpy()
cam_np = cam.cpu().numpy()

heatmap = cv2.applyColorMap((cam_np*255).astype(np.uint8), cv2.COLORMAP_JET)
heatmap = heatmap[..., ::-1] / 255.0

overlay = (0.5 * img_np + 0.5 * heatmap).clip(0,1)

plt.figure(figsize=(6,6))
plt.imshow(overlay)
plt.title('Grad-CAM Explanation')
plt.axis('off')
plt.show()

## ✅ Key Takeaway
- High-confidence predictions do **not guarantee faithful explanations**
- GradFaith-CAM quantitatively exposes shortcut learning